In [1]:
import torch.nn as nn
import torch
import os
from tqdm import tqdm
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
poems_text = open("yunus_emre.txt","r").read().lower()

all_letters = "".join(sorted(list(set(poems_text))))

print(f"{len(all_letters)} characters found!")

52 characters found!


In [3]:
def letter_to_index(letter: str):
    """converts letter to an index
    """
    return all_letters.find(letter.lower())

def letter_to_tensor(letter: str):
    tensor = torch.zeros(1, len(all_letters))
    tensor[0][letter_to_index(letter)] = 1
    return tensor

def line_to_tensor(line):
    tensor = torch.zeros(len(line),1,len(all_letters),dtype=torch.float32)
    for idx, letter in tqdm(enumerate(line)):
        tensor[idx][0][letter_to_index(letter)] = 1
    return tensor

In [4]:
class LSTM(nn.Module):

    def __init__(self, input_size:int,
                        hidden_size:int,
                        hidden_layers: int = 1):
        """Initialize an LSTM 
        for training purposes
        """
        super(LSTM,self).__init__()

        self.input_size = input_size
        self.hidden = None

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=hidden_layers,
            batch_first=True
        )

        self.h2o = nn.Linear(hidden_size, input_size)


    def forward(self, input):
        
        if self.hidden is None:
            output, self.hidden = self.lstm(input)
        else:
            output, self.hidden = self.lstm(input,self.hidden)

        self.hidden = tuple(item.detach() for item in self.hidden)
        output = self.h2o(output)

        return output

In [5]:
def train(rnn, inputs, optimizer,lr=0.005):
    
    
    rnn.train()
    loss_fn = nn.CrossEntropyLoss()
    losses = 0
    clipping_value = 1 # arbitrary value of your choosing
    rnn.hidden = None
    for idx in range(inputs.shape[0]-1):
        output = rnn(inputs[idx])
        loss = loss_fn(torch.flatten(output), torch.flatten(inputs[idx+1]))
        losses += loss
    
    #optimizer.zero_grad()
    loss.backward()
    #optimizer.step()
    torch.nn.utils.clip_grad_norm_(rnn.parameters(), clipping_value)
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-lr)

    return losses / (inputs.shape[0])

def train_other(rnn, 
                inputs, 
                optimizer, 
                min_seq_size=20, 
                max_seq_size=60):
    """
    """
    loss_fn = nn.CrossEntropyLoss()

    losses = 0
    rnn.train()
    pbar = tqdm(range(len(inputs) - max_seq_size))
    for idx in pbar:
        rnn.zero_grad()
        seq_len = random.randint(min_seq_size,max_seq_size)
        rnn.hidden = None
        input, target = inputs[idx:idx+seq_len], inputs[idx+1:idx+seq_len+1]
        output = rnn(input)

        loss = loss_fn(torch.flatten(output), torch.flatten(target))
        #loss = loss_fn(output, target)
        losses += loss
        loss.backward()
        #optimizer.step()
        for p in rnn.parameters():
            p.data.add_(p.grad.data, alpha=-0.005)

        if idx % 1000 == 0:
            pbar.set_description(f"Epoch: {idx}: {losses/idx:.2f}")
  
    return losses / idx 

In [6]:
input = line_to_tensor(poems_text).to(device)

rnn = LSTM(
    input_size=len(all_letters),
    hidden_size=512,
    hidden_layers=2
).to(device)
rnn.load_state_dict(torch.load("rnn.bin"))
optimizer = torch.optim.Adam(rnn.parameters(), amsgrad=True, lr=0.001)



114972it [00:00, 280589.15it/s]


In [7]:

epoch = 0
# while True:

#     loss = train_other(rnn, input, optimizer)

#     #print(f"Epoch {epoch} | Loss: {loss}")
#     epoch += 1
# while True:

#     pbar = tqdm(range(10000))
#     batch_size = random.randint(30,100) #100
#     batch_count = input.shape[0]//batch_size
#     loss_epoch = []
    
#     for epoch in pbar:

#         batch = int(random.random() * batch_count)
#         rnn.zero_grad()
#         inps = input[batch*batch_size: batch*batch_size + batch_size]
#         loss = train(rnn, inps, optimizer,lr=0.001)
#         loss_epoch.append(float(loss))
#         pbar.set_description(f"Epoch {epoch} | Batch Size: {batch_size} | Loss: {sum(loss_epoch)/len(loss_epoch):.2f}")

#         if epoch % 1000 == 0:
#         #         print(f"Epoch {epoch} Avg loss: {sum(loss_epoch)/len(loss_epoch):.2f}")
#                 loss_epoch = []

#     print(f"Epoch {epoch} Avg loss: {sum(loss_epoch)/len(loss_epoch):.2f}")
   

In [8]:
torch.save(rnn.state_dict(), "rnn.bin")
#rnn.load_state_dict(torch.load("rnn_2layer_512_softmax.bin"))

In [33]:

rnn.eval()

def tensor_to_char(tensor):
    return all_letters[int((tensor == 1).nonzero(as_tuple=True)[0])]

ch = "bir ben var"
for c in ch:
    output = letter_to_tensor(c).to(device)
    output = rnn(output)
    #print(c,end="")

print(ch,end="")
rnn.hidden = None
with torch.no_grad():
    for idx in range(200):

        output = rnn(output)
        output_dist = output.data.view(-1).div(0.9).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        #top_i = int(output.argmax())#
        print(all_letters[top_i],end="")
        output = letter_to_tensor(all_letters[top_i]).to(device)
        

bir ben variim canım allarım allah sana beni gelir



aşkın beni gelir




ene gelir bir gönül kalam ol ya kalın gönül gördüm elim bu yaşk ile derviş gelir




ene gelir bir dem bir gönül eyle derviş yana gelir
